#Linear regresor

In [ ]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import loguniform, uniform
from scipy.stats import uniform, randint

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_validate, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, BayesianRidge, SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.base import BaseEstimator, RegressorMixin

from skopt import BayesSearchCV
from skopt.space import Categorical, Real, Integer

In [ ]:
# Cargar el dataset
df = pd.read_csv('/content/AmesHousing.csv')  # Reemplaza con la ruta real

#Limpieza base de datos

##Numeric Features

In [ ]:
# --- Paso 1: Manejo de valores faltantes ---

# 1.1 Eliminar columnas con más del 65% de valores faltantes
missing_threshold = 0.65
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > missing_threshold].index
df = df.drop(columns=cols_to_drop)

# 1.2 Rellenar numéricas con la mediana
num_cols = df.select_dtypes(include=['number']).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

# 1.3 Rellenar categóricas con la moda
cat_cols = df.select_dtypes(include=['object', 'category', 'string']).columns
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)

# --- Paso 2: Separación de variables ---

# 2.1 Variables numéricas
X_num = df.select_dtypes(include=['number']).copy()
if 'SalePrice' not in X_num.columns:
    raise ValueError("La columna 'SalePrice' no se encuentra en las variables numéricas.")

# Calcular correlación con la variable objetivo
correlations = X_num.corr()['SalePrice']
relevant_num_features = correlations[(correlations.abs() > 0.25) & (correlations.index != 'SalePrice')].index

# Filtrar el dataframe numérico
X_num_filtered = X_num[relevant_num_features]

# 2.2 Variables categóricas
X_cat = df.select_dtypes(include=['object', 'category', 'string']).copy()

<ipython-input-4-c58525b68492>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
<ipython-input-4-c58525b68492>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

##Categorical features

In [ ]:
# Inicializamos contenedores
X_cat_binary = pd.DataFrame(index=df.index)
X_cat_ordinal = pd.DataFrame(index=df.index)

for col in X_cat.columns:
    unique_vals = X_cat[col].nunique()

    if unique_vals == 2:
        # --- Codificación binaria personalizada ---
        means = df.groupby(col)['SalePrice'].mean().sort_values()
        top_category = means.idxmax()
        encoding_map = {top_category: 1}
        encoded = X_cat[col].map(lambda x: encoding_map.get(x, 0))
        X_cat_binary[col + "_bin"] = encoded

        print(f"\n[Binary Encoding] {col}:")
        print(f"Categoría con mayor SalePrice: {top_category}")
        print(encoded.head())

    elif unique_vals > 2:
        # --- Ordinal Encoding basado en SalePrice promedio ---
        means = df.groupby(col)['SalePrice'].mean().sort_values()
        encoding_map = {cat: rank for rank, cat in enumerate(means.index, start=1)}
        encoded = X_cat[col].map(encoding_map)
        X_cat_ordinal[col + "_ord"] = encoded

        print(f"\n[Ordinal Encoding] {col}:")
        print(f"Mapeo de valores:\n{encoding_map}")
        print(encoded.head())



[Ordinal Encoding] MS Zoning:
Mapeo de valores:
{'A (agr)': 1, 'C (all)': 2, 'I (all)': 3, 'RM': 4, 'RH': 5, 'RL': 6, 'FV': 7}
0    6
1    5
2    6
3    6
4    6
Name: MS Zoning, dtype: int64

[Binary Encoding] Street:
Categoría con mayor SalePrice: Pave
0    1
1    1
2    1
3    1
4    1
Name: Street, dtype: int64

[Ordinal Encoding] Lot Shape:
Mapeo de valores:
{'Reg': 1, 'IR1': 2, 'IR3': 3, 'IR2': 4}
0    2
1    1
2    2
3    1
4    2
Name: Lot Shape, dtype: int64

[Ordinal Encoding] Land Contour:
Mapeo de valores:
{'Bnk': 1, 'Lvl': 2, 'Low': 3, 'HLS': 4}
0    2
1    2
2    2
3    2
4    2
Name: Land Contour, dtype: int64

[Ordinal Encoding] Utilities:
Mapeo de valores:
{'NoSewr': 1, 'NoSeWa': 2, 'AllPub': 3}
0    3
1    3
2    3
3    3
4    3
Name: Utilities, dtype: int64

[Ordinal Encoding] Lot Config:
Mapeo de valores:
{'FR2': 1, 'Inside': 2, 'Corner': 3, 'FR3': 4, 'CulDSac': 5}
0    3
1    2
2    3
3    3
4    2
Name: Lot Config, dtype: int64

[Ordinal Encoding] Land Slope:
Map

In [ ]:
# Unir numéricas relevantes + categóricas codificadas
X_final = pd.concat([X_num_filtered, X_cat_binary, X_cat_ordinal], axis=1)

# Añadir la columna objetivo
X_final['SalePrice'] = df['SalePrice']

# Resumen
print("\n=== DataFrame final ===")
print(f"Shape: {X_final.shape}")
print(X_final.info())


=== DataFrame final ===
Shape: (2930, 61)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 61 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Lot Frontage        2930 non-null   float64
 1   Lot Area            2930 non-null   int64  
 2   Overall Qual        2930 non-null   int64  
 3   Year Built          2930 non-null   int64  
 4   Year Remod/Add      2930 non-null   int64  
 5   Mas Vnr Area        2930 non-null   float64
 6   BsmtFin SF 1        2930 non-null   float64
 7   Total Bsmt SF       2930 non-null   float64
 8   1st Flr SF          2930 non-null   int64  
 9   2nd Flr SF          2930 non-null   int64  
 10  Gr Liv Area         2930 non-null   int64  
 11  Bsmt Full Bath      2930 non-null   float64
 12  Full Bath           2930 non-null   int64  
 13  Half Bath           2930 non-null   int64  
 14  TotRms AbvGrd       2930 non-null   int64  
 15  Fireplaces  

In [ ]:
# Seleccionar solo las variables categóricas codificadas (excluyendo SalePrice)
X_cat_encoded = X_final.drop(columns=X_num_filtered.columns.tolist() + ['SalePrice'])

# Calcular correlación con SalePrice
correlations = X_cat_encoded.corrwith(X_final['SalePrice'])

# Filtrar las que tienen |corr| > 0.25
relevant_cat_features = correlations[correlations.abs() > 0.25].index.tolist()

# Reconstruir X_final con:
# - numéricas relevantes (X_num_filtered)
# - categóricas codificadas relevantes
# - SalePrice (target)
X_final_filtered = pd.concat([
    X_num_filtered,
    X_cat_encoded[relevant_cat_features],
    X_final[['SalePrice']]
], axis=1)
print(X_final_filtered)

      Lot Frontage  Lot Area  Overall Qual  Year Built  Year Remod/Add  \
0            141.0     31770             6        1960            1960   
1             80.0     11622             5        1961            1961   
2             81.0     14267             6        1958            1958   
3             93.0     11160             7        1968            1968   
4             74.0     13830             5        1997            1998   
...            ...       ...           ...         ...             ...   
2925          37.0      7937             6        1984            1984   
2926          68.0      8885             5        1983            1983   
2927          62.0     10441             5        1992            1992   
2928          77.0     10010             5        1974            1975   
2929          74.0      9627             7        1993            1994   

      Mas Vnr Area  BsmtFin SF 1  Total Bsmt SF  1st Flr SF  2nd Flr SF  ...  \
0            112.0         639.

##Standard scaler

In [ ]:
# --- 1. Separar target ---
y = X_final_filtered['SalePrice']
X = X_final_filtered.drop(columns=['SalePrice'])

# --- 2. Identificar columnas one-hot (solo contienen 0 y 1) ---
one_hot_cols = [col for col in X.columns if set(X[col].unique()) <= {0, 1}]
non_one_hot_cols = [col for col in X.columns if col not in one_hot_cols]

# --- 3. Escalar solo las columnas no one-hot ---
scaler = StandardScaler()
X_scaled_part = pd.DataFrame(scaler.fit_transform(X[non_one_hot_cols]),
                             columns=non_one_hot_cols,
                             index=X.index)

# --- 4. Unir columnas escaladas con las one-hot intactas ---
X_final_scaled = pd.concat([X_scaled_part, X[one_hot_cols]], axis=1)

# --- 5. Añadir nuevamente la variable objetivo ---
X_final_scaled['SalePrice'] = y

# Verificación
print(f"X_final_scaled.shape: {X_final_scaled.shape}")

X_final_scaled.shape: (2930, 43)


##Outliers treatment

In [ ]:
# --- 1. Separar X e y ---
y = X_final_scaled['SalePrice']
X = X_final_scaled.drop(columns=['SalePrice'])

# --- 2. Columnas a excluir de outlier detection en X (ordinales y one-hot relevantes) ---
cols_excluir = X_cat_encoded[relevant_cat_features].columns.tolist()

# --- 3. Columnas numéricas puras donde aplicar IQR ---
cols_outlier_check = [col for col in X.columns if col not in cols_excluir]

# --- 4. Filtrado IQR en variables numéricas (X) ---
Q1_X = X[cols_outlier_check].quantile(0.25)
Q3_X = X[cols_outlier_check].quantile(0.75)
IQR_X = Q3_X - Q1_X
mask_X = ~((X[cols_outlier_check] < (Q1_X - 1.5 * IQR_X)) | (X[cols_outlier_check] > (Q3_X + 1.5 * IQR_X))).any(axis=1)

# --- 5. Filtrado IQR en el target (y) ---
Q1_y = y.quantile(0.25)
Q3_y = y.quantile(0.75)
IQR_y = Q3_y - Q1_y
mask_y = (y >= (Q1_y - 1.5 * IQR_y)) & (y <= (Q3_y + 1.5 * IQR_y))

# --- 6. Combinar ambas máscaras ---
final_mask = mask_X & mask_y

# --- 7. Filtrar datos finales ---
X_clean = X[final_mask]
y_clean = y[final_mask]
X_final_cleaned = pd.concat([X_clean, y_clean], axis=1)

# --- 8. Verificación ---
print(f"Original: {X.shape[0]} filas")
print(f"Después de quitar outliers en X: {mask_X.sum()} filas")
print(f"Después de quitar outliers en y: {mask_y.sum()} filas")
print(f"Filas finales después de ambos filtros: {final_mask.sum()} filas")


Original: 2930 filas
Después de quitar outliers en X: 2125 filas
Después de quitar outliers en y: 2793 filas
Filas finales después de ambos filtros: 2099 filas


#LinearRegresor

##Grid Search

In [ ]:
# === 1. Separar X e y ===
X = X_final_cleaned.drop(columns=['SalePrice'])
y = X_final_cleaned['SalePrice']

# === 2. Definir modelo base y parámetros (en LinearRegression hay pocos) ===
model = LinearRegression()
param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False]
}

# === 3. Métricas personalizadas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 4. Validación cruzada y GridSearch ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(model, param_grid, cv=cv, scoring=scoring, refit='R2', return_train_score=True)

# === 5. Entrenamiento ===
grid.fit(X, y)

# === 6. Resultados de la mejor combinación ===
best_model = grid.best_estimator_
print("Mejores hiperparámetros:", grid.best_params_)

# === 7. Validación cruzada con el mejor modelo ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

# === 8. Mostrar métricas promedio ===
print("\n=== Métricas promedio (5-fold CV) ===")
for metric in scoring:
    scores = results[f'test_{metric}']
    print(f"{metric}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

Mejores hiperparámetros: {'fit_intercept': True, 'positive': True}

=== Métricas promedio (5-fold CV) ===
MAE: 13512.9618 ± 318.4346
MSE: 321524363.7459 ± 31671930.7512
R2: 0.8899 ± 0.0090
MAPE: 0.0917 ± 0.0057


##Bayesian Optimization

In [ ]:
# Definir espacio de búsqueda
search_spaces = {
    'fit_intercept': Categorical([True, False]),
    'positive': Categorical([True, False])
}

# Optimizador bayesiano
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    scoring='r2',  # Usamos R2 como métrica principal
    cv=cv,
    n_iter=10,
    n_jobs=-1,
    refit=True,
    random_state=42
)

# Entrenamiento
opt.fit(X, y)

# Mejor modelo
best_model = opt.best_estimator_
print("Mejores hiperparámetros (BayesSearchCV):", opt.best_params_)

# Validación cruzada con el mejor modelo y métricas
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

# Resultados promedio
print("\n=== Métricas promedio (5-fold CV con BayesSearchCV) ===")
for metric in scoring:
    scores = results[f'test_{metric}']
    print(f"{metric}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

Mejores hiperparámetros (BayesSearchCV): OrderedDict([('fit_intercept', True), ('positive', False)])

=== Métricas promedio (5-fold CV con BayesSearchCV) ===
MAE: 13548.8639 ± 283.6849
MSE: 321805908.7574 ± 29636529.0437
R2: 0.8898 ± 0.0080
MAPE: 0.0925 ± 0.0057


#Lasso

##Grid Search

In [ ]:
# === 2. Definir modelo y espacio de búsqueda ===
model = Lasso(max_iter=10000)
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

# === 3. Definir métricas personalizadas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 4. Validación cruzada y búsqueda de hiperparámetros ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(model, param_grid, scoring='r2', cv=cv, refit=True, n_jobs=-1)

# === 5. Entrenamiento ===
grid.fit(X, y)

# === 6. Mejor modelo ===
best_model = grid.best_estimator_
print("Mejor alpha encontrado:", grid.best_params_['alpha'])

# === 7. Evaluación cruzada con métricas ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con Lasso) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")


Mejor alpha encontrado: 10.0

=== Métricas promedio (5-fold CV con Lasso) ===
MAE: 13550.3548 ± 282.6150
MSE: 321749439.2554 ± 29606567.0558
R2: 0.8899 ± 0.0080
MAPE: 0.0925 ± 0.0057


##Random Search

In [ ]:
# === 3. Espacio de búsqueda para alpha (escala logarítmica) ===
param_dist = {
    'alpha': loguniform(1e-4, 1e2)  # Entre 0.0001 y 100
}

# === 5. Configuración de validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Búsqueda aleatoria ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=20,  # Puedes ajustar el número de combinaciones aleatorias
    scoring='r2',
    cv=cv,
    random_state=42,
    n_jobs=-1
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejor alpha encontrado:", random_search.best_params_['alpha'])

# === 8. Evaluación con métricas personalizadas ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con RandomizedSearchCV Lasso) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejor alpha encontrado: 15.741890047456648

=== Métricas promedio (5-fold CV con RandomizedSearchCV Lasso) ===
MAE: 13551.7551 ± 282.4666
MSE: 321743353.7256 ± 29607108.0226
R2: 0.8899 ± 0.0080
MAPE: 0.0925 ± 0.0057


##Bayesian Optimization

In [ ]:
# === 3. Espacio de búsqueda para alpha (escala logarítmica)
search_spaces = {
    'alpha': Real(1e-4, 1e2, prior='log-uniform')
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Optimizador Bayesiano con Proceso Gaussiano ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=25,  # número de iteraciones de la búsqueda bayesiana
    cv=cv,
    scoring='r2',
    n_jobs=-1,
    refit=True,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejor alpha encontrado (BayesSearchCV):", opt.best_params_['alpha'])

# === 8. Evaluación con métricas personalizadas ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con BayesSearchCV Lasso) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejor alpha encontrado (BayesSearchCV): 13.316370182259133

=== Métricas promedio (5-fold CV con BayesSearchCV Lasso) ===
MAE: 13551.1413 ± 282.5111
MSE: 321742705.4293 ± 29604350.9970
R2: 0.8899 ± 0.0080
MAPE: 0.0925 ± 0.0057


#KernerRidge

##Grid Search

In [ ]:
# === 2. Definir modelo base ===
model = KernelRidge()

# === 3. Espacio de búsqueda para GridSearchCV ===
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3],           # solo aplica a kernel='poly'
    'gamma': [0.01, 0.1, 1]     # aplica a 'rbf' y 'poly'
}

# === 4. Definir métricas personalizadas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 5. Validación cruzada y búsqueda de hiperparámetros ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(model, param_grid, scoring='r2', cv=cv, n_jobs=-1, refit=True)

# === 6. Entrenamiento ===
grid.fit(X, y)

# === 7. Mejor modelo ===
best_model = grid.best_estimator_
print("Mejores hiperparámetros encontrados:")
print(grid.best_params_)

# === 8. Evaluación con métricas personalizadas ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con Kernel Ridge) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados:
{'alpha': 1, 'degree': 3, 'gamma': 0.01, 'kernel': 'poly'}

=== Métricas promedio (5-fold CV con Kernel Ridge) ===
MAE: 11617.8672 ± 359.1808
MSE: 256925191.2970 ± 24693271.0367
R2: 0.9120 ± 0.0074
MAPE: 0.0804 ± 0.0041


##Random Search

In [ ]:
# === 3. Espacio de búsqueda para RandomSearch ===
param_dist = {
    'alpha': uniform(1e-3, 1e2),  # valores entre 0.001 y 100
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': randint(2, 5),      # para 'poly'
    'gamma': uniform(0.001, 1)    # para 'rbf' y 'poly'
}

# === 4. Métricas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Random Search ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=cv,
    random_state=42,
    refit=True,
    n_jobs=-1
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejores hiperparámetros encontrados (RandomSearchCV):")
print(random_search.best_params_)

# === 8. Evaluación cruzada con el mejor modelo ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con Kernel Ridge + RandomSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (RandomSearchCV):
{'alpha': np.float64(43.93465018657701), 'degree': 4, 'gamma': np.float64(0.03242918568673425), 'kernel': 'poly'}

=== Métricas promedio (5-fold CV con Kernel Ridge + RandomSearchCV) ===
MAE: 13141.2932 ± 330.4325
MSE: 334689719.4144 ± 36766279.7762
R2: 0.8855 ± 0.0107
MAPE: 0.0892 ± 0.0054


##Bayesian Optimization

In [ ]:
# === 3. Espacio de búsqueda bayesiano ===
search_spaces = {
    'alpha': Real(1e-3, 1e2, prior='log-uniform'),
    'kernel': Categorical(['linear', 'rbf', 'poly']),
    'degree': Integer(2, 5),              # solo se usará si kernel == 'poly'
    'gamma': Real(1e-3, 1.0, prior='log-uniform')  # se usa en 'rbf' y 'poly'
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Optimizador bayesiano ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=30,  # puedes ajustar el número de iteraciones
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejores hiperparámetros encontrados (BayesSearchCV):")
print(opt.best_params_)

# === 8. Evaluación cruzada final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con Kernel Ridge + BayesSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (BayesSearchCV):
OrderedDict([('alpha', 0.01587616205787679), ('degree', 4), ('gamma', 0.0014035050621349604), ('kernel', 'rbf')])

=== Métricas promedio (5-fold CV con Kernel Ridge + BayesSearchCV) ===
MAE: 11778.2662 ± 342.6849
MSE: 259738037.6197 ± 23466436.8034
R2: 0.9110 ± 0.0069
MAPE: 0.0811 ± 0.0038


#SGDRegressor

##Grid Search

In [ ]:
# === 2. Modelo base ===
model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)

# === 3. Espacio de búsqueda (híperparámetros más relevantes para regresión) ===
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'loss': ['squared_error', 'huber'],
    'l1_ratio': [0.15, 0.5, 0.9]  # solo usado cuando penalty='elasticnet'
}

# === 4. Métricas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Grid Search ===
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=cv,
    n_jobs=-1,
    refit=True
)

# === 7. Entrenamiento ===
grid.fit(X, y)
best_model = grid.best_estimator_

print("Mejores hiperparámetros encontrados (GridSearchCV):")
print(grid.best_params_)

# === 8. Evaluación final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con SGDRegressor + GridSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (GridSearchCV):
{'alpha': 0.001, 'l1_ratio': 0.15, 'loss': 'squared_error', 'penalty': 'l2'}

=== Métricas promedio (5-fold CV con SGDRegressor + GridSearchCV) ===
MAE: 13536.8430 ± 310.4108
MSE: 323917696.0276 ± 30801618.1587
R2: 0.8891 ± 0.0085
MAPE: 0.0923 ± 0.0059


##Random Search

In [ ]:
# === 3. Espacio de búsqueda aleatorio ===
param_dist = {
    'alpha': loguniform(1e-5, 1e-1),
    'penalty': ['l2', 'l1', 'elasticnet'],
    'loss': ['squared_error', 'huber'],
    'l1_ratio': uniform(0.1, 0.9)  # solo se usa cuando penalty = 'elasticnet'
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Random Search ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=cv,
    n_jobs=-1,
    refit=True,
    random_state=42
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejores hiperparámetros encontrados (RandomizedSearchCV):")
print(random_search.best_params_)

# === 8. Evaluación cruzada ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con SGDRegressor + RandomizedSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (RandomizedSearchCV):
{'alpha': np.float64(0.0024400607090817534), 'l1_ratio': np.float64(0.501249477568232), 'loss': 'squared_error', 'penalty': 'elasticnet'}

=== Métricas promedio (5-fold CV con SGDRegressor + RandomizedSearchCV) ===
MAE: 13535.9260 ± 310.6520
MSE: 323861259.8436 ± 30792864.2135
R2: 0.8891 ± 0.0085
MAPE: 0.0923 ± 0.0059


##Bayesian Optimization

In [ ]:
# === 3. Espacio de búsqueda bayesiano ===
search_spaces = {
    'alpha': Real(1e-5, 1e-1, prior='log-uniform'),
    'penalty': Categorical(['l2', 'l1', 'elasticnet']),
    'loss': Categorical(['squared_error', 'huber']),
    'l1_ratio': Real(0.1, 1.0)  # solo se usa si penalty = 'elasticnet'
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Optimizador bayesiano ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=30,
    scoring='r2',
    cv=cv,
    n_jobs=-1,
    refit=True,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejores hiperparámetros encontrados (BayesSearchCV):")
print(opt.best_params_)

# === 8. Evaluación final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con SGDRegressor + BayesSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-05, 0.1, np.str_('squared_error'), np.str_('l1')] before, using random point [4.554490657921972e-05, 0.24318167559373566, 'squared_error', 'elasticnet']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-05, 0.1, np.str_('squared_error'), np.str_('l1')] before, using random point [0.010642995574088522, 0.4350325730222441, 'squared_error', 'l2']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-05, 0.1, np.str_('squared_error'), np.str_('l1')] before, using random point [0.00029641347504855977, 0.2679481967517265, 'squared_error', 'elasticnet']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective 

Mejores hiperparámetros encontrados (BayesSearchCV):
OrderedDict([('alpha', 0.0029397976202716882), ('l1_ratio', 0.7978279409450943), ('loss', 'squared_error'), ('penalty', 'elasticnet')])

=== Métricas promedio (5-fold CV con SGDRegressor + BayesSearchCV) ===
MAE: 13538.5982 ± 309.9608
MSE: 324027421.3332 ± 30818486.8127
R2: 0.8891 ± 0.0085
MAPE: 0.0923 ± 0.0059


#BayesianRidge

##Grid Search

In [ ]:
# === 2. Modelo base ===
model = BayesianRidge()

# === 3. Espacio de búsqueda para hiperparámetros ===
param_grid = {
    'alpha_1': [1e-6, 1e-5, 1e-4],  # hiperparámetros del prior Gamma
    'alpha_2': [1e-6, 1e-5, 1e-4],
    'lambda_1': [1e-6, 1e-5, 1e-4],
    'lambda_2': [1e-6, 1e-5, 1e-4]
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Grid Search ===
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1
)

# === 7. Entrenamiento ===
grid.fit(X, y)
best_model = grid.best_estimator_

print("Mejores hiperparámetros encontrados (GridSearchCV):")
print(grid.best_params_)

# === 8. Evaluación final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con BayesianRidge + GridSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (GridSearchCV):
{'alpha_1': 0.0001, 'alpha_2': 1e-06, 'lambda_1': 1e-06, 'lambda_2': 1e-06}

=== Métricas promedio (5-fold CV con BayesianRidge + GridSearchCV) ===
MAE: 13540.9658 ± 292.1413
MSE: 321545961.2378 ± 29652221.2776
R2: 0.8899 ± 0.0082
MAPE: 0.0923 ± 0.0057


##Random Search

In [ ]:
# === 3. Espacio de búsqueda aleatoria ===
param_dist = {
    'alpha_1': loguniform(1e-7, 1e-3),
    'alpha_2': loguniform(1e-7, 1e-3),
    'lambda_1': loguniform(1e-7, 1e-3),
    'lambda_2': loguniform(1e-7, 1e-3)
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Randomized Search ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1,
    random_state=42
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejores hiperparámetros encontrados (RandomizedSearchCV):")
print(random_search.best_params_)

# === 8. Evaluación final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con BayesianRidge + RandomizedSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (RandomizedSearchCV):
{'alpha_1': np.float64(0.0003538758864779237), 'alpha_2': np.float64(7.74211647399625e-06), 'lambda_1': np.float64(3.008686821445843e-07), 'lambda_2': np.float64(7.128188058401354e-05)}

=== Métricas promedio (5-fold CV con BayesianRidge + RandomizedSearchCV) ===
MAE: 13540.9658 ± 292.1412
MSE: 321545961.2383 ± 29652221.2518
R2: 0.8899 ± 0.0082
MAPE: 0.0923 ± 0.0057


##Bayesian Optimization

In [ ]:
# === 3. Espacio de búsqueda bayesiano ===
search_spaces = {
    'alpha_1': Real(1e-7, 1e-3, prior='log-uniform'),
    'alpha_2': Real(1e-7, 1e-3, prior='log-uniform'),
    'lambda_1': Real(1e-7, 1e-3, prior='log-uniform'),
    'lambda_2': Real(1e-7, 1e-3, prior='log-uniform')
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Optimizador bayesiano ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=30,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejores hiperparámetros encontrados (BayesSearchCV):")
print(opt.best_params_)

# === 8. Evaluación cruzada final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con BayesianRidge + BayesSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (BayesSearchCV):
OrderedDict([('alpha_1', 0.0008992531372518143), ('alpha_2', 1.064391001430717e-07), ('lambda_1', 8.571681995566809e-07), ('lambda_2', 8.31024835273467e-06)])

=== Métricas promedio (5-fold CV con BayesianRidge + BayesSearchCV) ===
MAE: 13540.9658 ± 292.1412
MSE: 321545961.2395 ± 29652221.2040
R2: 0.8899 ± 0.0082
MAPE: 0.0923 ± 0.0057


#GaussianProcessRegressor

##Grid Search

In [ ]:
# === 2. Definir kernels combinables para búsqueda ===
kernel_1 = ConstantKernel(1.0) * RBF(length_scale=1.0)
kernel_2 = ConstantKernel(1.0) * DotProduct()
kernel_3 = ConstantKernel(1.0) * RBF(length_scale=1.0) + WhiteKernel()

# === 3. Espacio de búsqueda ===
param_grid = {
    'kernel': [kernel_1, kernel_2, kernel_3],
    'alpha': [1e-10, 1e-5, 1e-2]
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Grid Search con GPR ===
model = GaussianProcessRegressor(normalize_y=True)
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=cv,
    n_jobs=-1,
    refit=True
)

# === 7. Entrenamiento ===
grid.fit(X, y)
best_model = grid.best_estimator_

print("Mejores hiperparámetros encontrados (GridSearchCV):")
print(best_model.kernel)
print("Alpha:", grid.best_params_['alpha'])

# === 8. Evaluación cruzada ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con GPR + GridSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (GridSearchCV):
1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1)
Alpha: 0.01

=== Métricas promedio (5-fold CV con GPR + GridSearchCV) ===
MAE: 11791.1221 ± 368.5425
MSE: 262043397.5834 ± 21577384.1793
R2: 0.9102 ± 0.0067
MAPE: 0.0816 ± 0.0034


##Random Search

In [ ]:
# === 3. Espacio de búsqueda aleatorio ===
param_dist = {
    'kernel': [
        ConstantKernel(1.0, (0.1, 10.0)) * RBF(length_scale=1.0),
        ConstantKernel(1.0, (0.1, 10.0)) * DotProduct(),
        ConstantKernel(1.0, (0.1, 10.0)) * RBF(length_scale=1.0) + WhiteKernel()
    ],
    'alpha': loguniform(1e-10, 1e-2)
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Randomized Search ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=15,  # puedes aumentar según tiempo disponible
    scoring='r2',
    cv=cv,
    n_jobs=-1,
    refit=True,
    random_state=42
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejores hiperparámetros encontrados (RandomizedSearchCV):")
print(best_model.kernel)
print("Alpha:", random_search.best_params_['alpha'])

# === 8. Evaluación cruzada ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con GPR + RandomizedSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (RandomizedSearchCV):
1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1)
Alpha: 4.452048365748848e-07

=== Métricas promedio (5-fold CV con GPR + RandomizedSearchCV) ===
MAE: 11791.1210 ± 368.5415
MSE: 262043364.0511 ± 21577430.0651
R2: 0.9102 ± 0.0067
MAPE: 0.0816 ± 0.0034


##Bayesian Optimization

In [ ]:
# === 1. Función para construir el kernel según nombre ===
def build_kernel(name):
    if name == 'rbf':
        return ConstantKernel(1.0, (0.1, 10.0)) * RBF(length_scale=1.0)
    elif name == 'dot':
        return ConstantKernel(1.0, (0.1, 10.0)) * DotProduct()
    elif name == 'rbf+white':
        return ConstantKernel(1.0, (0.1, 10.0)) * RBF(length_scale=1.0) + WhiteKernel()
    else:
        raise ValueError("Nombre de kernel no reconocido")

# === 2. Estimador personalizado para usar con BayesSearchCV ===
class GPRKernelWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, kernel_name='rbf', alpha=1e-10):
        self.kernel_name = kernel_name
        self.alpha = alpha
        self.model = None

    def fit(self, X, y):
        kernel = build_kernel(self.kernel_name)
        self.model = GaussianProcessRegressor(kernel=kernel, alpha=self.alpha)
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

    @property
    def kernel(self):
        return self.model.kernel_ if self.model is not None else None

# === 3. Espacio de búsqueda para BayesSearchCV ===
search_spaces = {
    'kernel_name': Categorical(['rbf', 'dot', 'rbf+white']),
    'alpha': Real(1e-10, 1e-2, prior='log-uniform')
}

# === 4. Modelo base ===
model = GPRKernelWrapper()

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Configurar BayesSearchCV ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=25,  # puedes aumentar para una búsqueda más exhaustiva
    scoring='r2',
    cv=cv,
    n_jobs=-1,
    refit=True,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejores hiperparámetros encontrados (BayesSearchCV):")
print(best_model.kernel)
print("Alpha:", opt.best_params_['alpha'])

# === 8. Evaluación con métricas ===
scoring = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con GPR + BayesSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-10, np.str_('dot')] before, using random point [2.900170356212243e-10, 'rbf+white']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-10, np.str_('dot')] before, using random point [2.771959065755811e-08, 'rbf']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.01, np.str_('dot')] before, using random point [0.0003396918377558348, 'rbf+white']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.01, np.str_('dot')] before, using random point [4.0189876087203675e-08, 'rbf']
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/optimizer/optimizer.py:517: Use

Mejores hiperparámetros encontrados (BayesSearchCV):
1**2 * DotProduct(sigma_0=1)
Alpha: 0.0002491308632420524

=== Métricas promedio (5-fold CV con GPR + BayesSearchCV) ===
r2: 0.8898 ± 0.0080
neg_mean_absolute_error: -13548.8598 ± 283.6893
neg_mean_squared_error: -321805873.1864 ± 29636598.4639


#RandomForestRegressor

##Grid Search

In [ ]:
# === 2. Modelo base ===
model = RandomForestRegressor(random_state=42, n_jobs=-1)

# === 3. Espacio de búsqueda para GridSearch ===
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

# === 4. Métricas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Grid Search ===
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1
)

# === 7. Entrenamiento ===
grid.fit(X, y)
best_model = grid.best_estimator_

print("Mejores hiperparámetros encontrados (GridSearchCV):")
print(grid.best_params_)

# === 8. Evaluación cruzada ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con RandomForestRegressor + GridSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/util

Mejores hiperparámetros encontrados (GridSearchCV):
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}

=== Métricas promedio (5-fold CV con RandomForestRegressor + GridSearchCV) ===
MAE: 12282.5522 ± 646.1886
MSE: 303178936.3572 ± 49042575.5361
R2: 0.8962 ± 0.0161
MAPE: 0.0863 ± 0.0079


##Random Search

In [ ]:
# === 3. Espacio de búsqueda aleatoria ===
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': [None] + list(range(5, 31, 5)),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['auto', 'sqrt', 'log2']
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Randomized Search ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=cv,
    refit=True,
    random_state=42,
    n_jobs=-1
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejores hiperparámetros encontrados (RandomizedSearchCV):")
print(random_search.best_params_)

# === 8. Evaluación cruzada final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con RandomForestRegressor + RandomizedSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
45 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

Mejores hiperparámetros encontrados (RandomizedSearchCV):
{'max_depth': 25, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 445}

=== Métricas promedio (5-fold CV con RandomForestRegressor + RandomizedSearchCV) ===
MAE: 12387.7479 ± 642.9480
MSE: 309167441.5961 ± 49391028.6698
R2: 0.8942 ± 0.0161
MAPE: 0.0872 ± 0.0079


##Bayesian Optimization

In [ ]:
# === 3. Espacio de búsqueda bayesiano ===
search_spaces = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Categorical(['sqrt', 'log2'])
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. BayesSearchCV con GPR ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=30,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejores hiperparámetros encontrados (BayesSearchCV):")
print(opt.best_params_)

# === 8. Evaluación cruzada ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con RandomForestRegressor + BayesSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (BayesSearchCV):
OrderedDict([('max_depth', 30), ('max_features', 'sqrt'), ('min_samples_leaf', 1), ('min_samples_split', 2), ('n_estimators', 500)])

=== Métricas promedio (5-fold CV con RandomForestRegressor + BayesSearchCV) ===
MAE: 12199.0059 ± 624.9526
MSE: 301186327.0232 ± 47941241.3479
R2: 0.8969 ± 0.0157
MAPE: 0.0858 ± 0.0075


#Support Vector Machines Regressor

##Grid Search

In [ ]:
# === 2. Modelo base ===
model = SVR()

# === 3. Espacio de búsqueda para GridSearch ===
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 1],
    'gamma': ['scale', 'auto']
}

# === 4. Métricas personalizadas ===
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
    'MAPE': make_scorer(mean_absolute_percentage_error)
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Grid Search ===
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1
)

# === 7. Entrenamiento ===
grid.fit(X, y)
best_model = grid.best_estimator_

print("Mejores hiperparámetros encontrados (GridSearchCV):")
print(grid.best_params_)

# === 8. Evaluación cruzada ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con SVR + GridSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (GridSearchCV):
{'C': 100, 'epsilon': 1, 'gamma': 'scale', 'kernel': 'linear'}

=== Métricas promedio (5-fold CV con SVR + GridSearchCV) ===
MAE: 13659.2587 ± 461.4721
MSE: 355259403.3134 ± 43438066.6802
R2: 0.8783 ± 0.0138
MAPE: 0.0905 ± 0.0055


##Random Search

In [ ]:
# === 3. Espacio de búsqueda aleatoria ===
param_dist = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': uniform(0.1, 100),       # valores entre 0.1 y 100.1
    'epsilon': uniform(0.01, 1),  # valores entre 0.01 y 1.01
    'gamma': ['scale', 'auto']
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. Randomized Search ===
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1,
    random_state=42
)

# === 7. Entrenamiento ===
random_search.fit(X, y)
best_model = random_search.best_estimator_

print("Mejores hiperparámetros encontrados (RandomizedSearchCV):")
print(random_search.best_params_)

# === 8. Evaluación cruzada final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con SVR + RandomizedSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (RandomizedSearchCV):
{'C': np.float64(82.97375091519292), 'epsilon': np.float64(0.3667533266935893), 'gamma': 'auto', 'kernel': 'linear'}

=== Métricas promedio (5-fold CV con SVR + RandomizedSearchCV) ===
MAE: 13742.0440 ± 476.4561
MSE: 362025061.2933 ± 45000976.2265
R2: 0.8760 ± 0.0143
MAPE: 0.0907 ± 0.0055


##Bayesian Optimization

In [ ]:
# === 3. Espacio de búsqueda bayesiano ===
search_spaces = {
    'kernel': Categorical(['linear', 'rbf', 'poly']),
    'C': Real(0.1, 100.0, prior='log-uniform'),
    'epsilon': Real(0.01, 1.0, prior='log-uniform'),
    'gamma': Categorical(['scale', 'auto'])
}

# === 5. Validación cruzada ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# === 6. BayesSearchCV ===
opt = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=30,
    scoring='r2',
    cv=cv,
    refit=True,
    n_jobs=-1,
    random_state=42
)

# === 7. Entrenamiento ===
opt.fit(X, y)
best_model = opt.best_estimator_

print("Mejores hiperparámetros encontrados (BayesSearchCV):")
print(opt.best_params_)

# === 8. Evaluación final ===
results = cross_validate(best_model, X, y, cv=cv, scoring=scoring)

print("\n=== Métricas promedio (5-fold CV con SVR + BayesSearchCV) ===")
for metric in scoring:
    print(f"{metric}: {np.mean(results[f'test_{metric}']):.4f} ± {np.std(results[f'test_{metric}']):.4f}")

Mejores hiperparámetros encontrados (BayesSearchCV):
OrderedDict([('C', 100.0), ('epsilon', 1.0), ('gamma', 'scale'), ('kernel', 'linear')])

=== Métricas promedio (5-fold CV con SVR + BayesSearchCV) ===
MAE: 13659.2587 ± 461.4721
MSE: 355259403.3134 ± 43438066.6802
R2: 0.8783 ± 0.0138
MAPE: 0.0905 ± 0.0055
